In [1]:
# *------- Basic setup -------*
import numpy as np
import pandas as pd
import os, random, time
import copy
from tqdm.notebook import tqdm
from multiprocessing import cpu_count
import matplotlib.pyplot as plt
from PIL import Image

# *------- torch -------*
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import torchvision
#import torchvision.transforms as transforms
from torchsummary import summary

# *------- albumentations -------*
#!pip install albumentations==1.0.3
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

# *------- sklearn -------*
from sklearn.model_selection import train_test_split

# *------- path -------*
base_path = "./"

# test 폴더를 한 단계 상위 폴더로 옮겨서 사용했습니다.
# 옮기지 않은 경우
#test_path = os.path.join(base_path, "test","test")
test_path = os.path.join(base_path, "test")
train_path = os.path.join(base_path, "train")

In [75]:
def set_seed(seed):
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_seed(1010)

DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
IMG_SIZE = (28,28)
BATCH_SIZE = 32
LEARNING_RATE = 0.001
NUM_CLASSES = 10
NUM_EPOCHS = 50
NUM_CPU = cpu_count()

In [17]:
def get_train_transforms():
    return A.Compose([
        A.HorizontalFlip(p=0.5),
        A.OneOf([A.Rotate(limit=10),
                 A.RandomBrightness(),
                 A.CoarseDropout(),
                 A.Cutout(num_holes=8, max_h_size=1, max_w_size=1, fill_value=1),
                 ], p=1.0),
        ToTensorV2(p=1.0)
    ])
def get_valid_transforms():
    return ToTensorV2(p=1.0)

def get_inferecne_transforms():
    return ToTensorV2(p=1.0)

In [18]:
class MnistDataset(Dataset):
    def __init__(self, X=None, y=None, transforms=None):
        super().__init__()
        self.X = X
        self.y = y
        self.transforms = transforms

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        img_path = self.X[idx]
        img = Image.open(img_path).convert("RGB")
        img = np.array(img, dtype=np.float32)
        img /= 255
        
        if self.transforms:
            img = self.transforms(image=img)['image']
        
        if self.y is not None:
            label = self.y[idx]
            label = torch.tensor(label, dtype=torch.int64)
            return img, label
        else:
            return img

In [205]:
tr_csv = pd.read_csv(os.path.join(base_path, "train","train_data.csv"))

tr_csv['path'] = tr_csv['filen_name'].apply(
    lambda x: os.path.join(train_path,x))


In [206]:
tr_csv

,filen_name,label,path
0,train0001.png,8,./train\train0001.png
1,train0002.png,8,./train\train0002.png
2,train0003.png,8,./train\train0003.png
3,train0004.png,8,./train\train0004.png
4,train0005.png,8,./train\train0005.png
...,...,...,...
4995,train4996.png,6,./train\train4996.png
4996,train4997.png,6,./train\train4997.png
4997,train4998.png,6,./train\train4998.png
4998,train4999.png,6,./train\train4999.png


In [207]:
teach_csv = tr_csv.sample(replace=False,n=3000)
teach_index = teach_csv.index

In [208]:
tr_csv = tr_csv.drop(teach_index).sample(frac=1)


In [210]:
train_x, train_y = tr_csv['path'].values, tr_csv["label"].values

In [211]:
train_x.shape, train_y.shape

((2000,), (2000,))

In [212]:
teach_x, teach_y = teach_csv['path'].values, teach_csv["label"].values

In [142]:
train_ds = MnistDataset(train_x, train_y, get_train_transforms())
valid_ds = MnistDataset(valid_x, valid_y, get_valid_transforms())

train_size = len(train_ds)
val_size = len(valid_ds)

train_dl = DataLoader(train_ds, batch_size = BATCH_SIZE, shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size = BATCH_SIZE, shuffle=False)


C:\ProgramData\Anaconda3\lib\site-packages\albumentations\augmentations\transforms.py:1744: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\albumentations\augmentations\transforms.py:688: FutureWarning: This class has been deprecated. Please use CoarseDropout
  warnings.warn(


In [130]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3) # 합성곱 연산 (입력 채널 수: 3, 출력 채널 수: 6, 필터 크기: 5x5, stride=1(default))
        self.pool1 = nn.MaxPool2d(2,2) # 합성곱 연산 (필터크기 2x2, stride=2)
        self.conv2 = nn.Conv2d(16, 16, 3) # 합성곱 연산 (입력 채널 수: 6, 출력 채널수: 16, 필터 크기: 5x5, stride=1(default))
        self.pool2 = nn.MaxPool2d(2, 2) # 합성곱 연산 (필터크기 2x2, stride=2)
        self.fc1 = nn.Linear(400,100) # 5x5 피쳐맵 16개를 일렬로 피면 16*5*5개의 노드가 생성됨.
        self.fc2 = nn.Linear(100, 10)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x))) # conv1 -> ReLU -> pool
        #print(x.shape)
        x = self.pool2(F.relu(self.conv2(x))) # conv2 -> ReLU -> pool2
       # print(x.shape)
        x = x.view(-1, 16*5*5) # 5x5 피쳐맵 16개를 일렬로 만든다.
       # print(x.shape)
        x = F.relu(self.fc1(x))
        #print(x.shape)
        x = F.relu(self.fc2(x))

        return x
net = Net().to(DEVICE)

In [131]:
from torchsummary import summary
net.to(DEVICE)
summary(net, (3,28,28), device = DEVICE.type)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             448
         MaxPool2d-2           [-1, 16, 13, 13]               0
            Conv2d-3           [-1, 16, 11, 11]           2,320
         MaxPool2d-4             [-1, 16, 5, 5]               0
            Linear-5                  [-1, 100]          40,100
            Linear-6                   [-1, 10]           1,010
Total params: 43,878
Trainable params: 43,878
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 0.12
Params size (MB): 0.17
Estimated Total Size (MB): 0.30
----------------------------------------------------------------


In [132]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=1e-5)
net.to(DEVICE)
param = list(net.parameters())


In [137]:
def train_model(model, criterion, optimizer, num_epochs, train_loader,val_loader,unlabeld_loader):
    since = time.time()
    best_model = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch + 1, num_epochs))
        
        # train
        model.train()
        running_loss = 0.0
        running_corrects = 0

        
        for step, (inputs, labels) in tqdm(enumerate(train_loader)):
            inputs = inputs.to(DEVICE)
            labels = labels.to(DEVICE)
            optimizer.zero_grad()
            with torch.set_grad_enabled(True):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
        epoch_loss = running_loss / train_size
        epoch_acc = running_corrects.double() / train_size
        print('Train Loss: {:.4f} Train Acc: {:.4f}'.format(epoch_loss, epoch_acc))
        
        # validate
        model.eval()
        running_loss = 0.0
        running_corrects = 0
        for inputs, labels in val_loader:
            inputs = inputs.to(DEVICE)
            labels = labels.to(DEVICE)
            optimizer.zero_grad()
            with torch.set_grad_enabled(False):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels) # labeld dataset
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
        epoch_loss = running_loss / val_size
        epoch_acc = running_corrects.double() / val_size
        print('Val Loss: {:.4f} Val Acc: {:.4f}'.format(epoch_loss, epoch_acc))
        print('-' * 30)
        if epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model = copy.deepcopy(model.state_dict())
        
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best Val Acc: {:.4f}'.format(best_acc))
    model.load_state_dict(best_model)
    return model

# train the model
model = train_model(net, criterion, optimizer, NUM_EPOCHS, train_dl, valid_dl)

3it [00:00, 21.21it/s]

Epoch 1/50


32it [00:01, 21.48it/s]


Train Loss: 2.0680 Train Acc: 0.4030


3it [00:00, 21.21it/s]

...

Train Loss: 1.7361 Train Acc: 0.4560


3it [00:00, 22.48it/s]

Val Loss: 1.5488 Val Acc: 0.5460
------------------------------
Epoch 50/50


32it [00:01, 22.76it/s]


Train Loss: 1.7242 Train Acc: 0.4600
Val Loss: 1.5437 Val Acc: 0.5490
------------------------------
Training complete in 1m 39s
Best Val Acc: 0.5490


In [159]:
teach_ds = MnistDataset(teach_csv['path'].values, transforms = get_inferecne_transforms())
teach_dl = DataLoader(teach_ds, batch_size = BATCH_SIZE, shuffle=False)

In [160]:
predictions = []

model.eval()
with torch.no_grad():
    for step, input in tqdm(enumerate(teach_dl), total=len(teach_dl)):
        input = input.to(DEVICE)
        y_pred = model(input).detach().cpu().numpy().argmax(axis=1).astype(int)
        
        predictions.extend(y_pred)

100%|██████████████████████████████████████████████████████████████████████████████████| 94/94 [00:01<00:00, 63.32it/s]


In [162]:
teach_csv['label'] = predictions
teach_csv

,filen_name,label,path
4169,train4170.png,2,./train\train4170.png
1989,train1990.png,1,./train\train1990.png
1451,train1452.png,6,./train\train1452.png
4844,train4845.png,3,./train\train4845.png
3951,train3952.png,9,./train\train3952.png
...,...,...,...
155,train0156.png,3,./train\train0156.png
437,train0438.png,1,./train\train0438.png
627,train0628.png,2,./train\train0628.png
4702,train4703.png,1,./train\train4703.png


In [179]:
cat_csv = pd.concat([tr_csv,teach_csv])

In [181]:
train_x, valid_x, train_y, valid_y = train_test_split(
    cat_csv['path'].values, cat_csv["label"].values, test_size=0.2, shuffle=True)

train_ds = MnistDataset(train_x, train_y, get_train_transforms())
valid_ds = MnistDataset(valid_x, valid_y, get_valid_transforms())

train_size = len(train_ds)
val_size = len(valid_ds)

train_dl = DataLoader(train_ds, batch_size = BATCH_SIZE, shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size = BATCH_SIZE, shuffle=False)

C:\ProgramData\Anaconda3\lib\site-packages\albumentations\augmentations\transforms.py:1744: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\albumentations\augmentations\transforms.py:688: FutureWarning: This class has been deprecated. Please use CoarseDropout
  warnings.warn(


In [182]:
model = train_model(net, criterion, optimizer, NUM_EPOCHS, train_dl, valid_dl)

2it [00:00, 18.72it/s]

Epoch 1/50


125it [00:06, 20.34it/s]


Train Loss: 1.3593 Train Acc: 0.6650


3it [00:00, 23.00it/s]

...

Train Loss: 1.0432 Train Acc: 0.6930


2it [00:00, 19.07it/s]

Val Loss: 0.8286 Val Acc: 0.7810
------------------------------
Epoch 50/50


125it [00:06, 20.32it/s]


Train Loss: 1.0305 Train Acc: 0.6993
Val Loss: 0.8325 Val Acc: 0.7720
------------------------------
Training complete in 6m 13s
Best Val Acc: 0.7810


In [130]:
ts_csv = pd.read_csv(os.path.join(base_path, "test","test_data.csv"))

ts_csv['path'] = ts_csv['file_name'].apply(
    lambda x: os.path.join(test_path, x))

test_x = ts_csv['path'].values
test_ds = MnistDataset(test_x, transforms = get_inferecne_transforms())

test_dl = DataLoader(test_ds, batch_size = BATCH_SIZE, shuffle=False, num_workers = NUM_CPU)

100%|████████████████████████████████████████████████████████████████████████████████| 157/157 [00:02<00:00, 72.49it/s]


In [131]:
preds

array([1, 0, 9, ..., 3, 9, 3], dtype=int64)

In [ ]:
submission = pd.read_csv('./sample_submission.csv') # sample submission 불러오기

submission['label'] = preds

submission.to_csv('submission.csv', index=False)